In [ ]:
import urllib.request
import zipfile
from pathlib import Path
from tqdm import tqdm

from kilosort.utils import DOWNLOADS_DIR

# NOTE: Be sure to update this filepath if you want the data downloaded to
#       a specific location.
SAVE_PATH = Path('/home/ubuntu/codespace/sorter-script/data') / 'ZFM-02370_mini.imec0.ap.short.bin'

class DownloadProgressBar(tqdm):
    """ from https://stackoverflow.com/a/53877507 """
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(url, output_path):
    # Download zip-compressed data file.
    zip_file = Path(output_path).with_suffix('.zip')
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=zip_file, reporthook=t.update_to)
    # Unzip to specified `output_path`.
    with zipfile.ZipFile(zip_file, "r") as zip_ref:
        zip_ref.extractall(output_path.parent)
    # Remove zip archive after unzipping.
    zip_file.unlink()


## CROPPED DATASET
URL = 'https://osf.io/download/67effd64f74150d8738b7f34/'
download_url(URL, SAVE_PATH)

In [4]:
from spikeinterface.sorters import run_sorter
import spikeinterface.full as si
import probeinterface as pi
import scipy
from pathlib import Path
import numpy as np

# --- Load Open Ephys recording ---
base_folder = Path('/home/ubuntu/codespace/SpikeInterface Dataset Tutorial')
oe_folder = base_folder / 'openephys_recording/2023-08-23_15-56-05'
rec = si.read_openephys(oe_folder, stream_id="0")

# --- Load probe and MAP its contacts to device channel indices ---
probe = pi.get_probe('cambridgeneurotech', 'ASSY-236-H5')

n_rec = rec.get_num_channels()
n_probe = probe.get_contact_count()

if n_probe != n_rec:
    raise ValueError(f"Probe contacts ({n_probe}) != recording channels ({n_rec}). "
                     f"Pick the correct probe variant or subset/remap accordingly.")

# Map 1:1 in current channel order
dev_inds = rec.ids_to_indices(rec.channel_ids)  # typically 0..N-1 in current order
probe.set_device_channel_indices(dev_inds)

# (optional) sanity check locations exist on the probe
assert probe.contact_positions is not None and probe.contact_positions.shape[0] == n_probe

# --- Attach probe (now it has device_channel_indices) ---
rec = rec.set_probe(probe, in_place=False)

# Quick check
print("Channel locations:", rec.get_channel_locations().shape)  # (N, 2)

# --- Run Kilosort4 ---
sorting_KS4 = run_sorter(
    sorter_name="kilosort4",
    recording=rec,
    folder="kilosort4_output",
    verbose=True,
)


Channel locations: (64, 2)


write_binary_recording (no parallelization):   0%|          | 0/300 [00:00<?, ?it/s]

write_binary_recording (no parallelization): 100%|██████████| 300/300 [00:07<00:00, 38.20it/s]
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: N samples: 9000000
kilosort.run_kilosort: N seconds: 300.0
kilosort.run_kilosort: N batches: 150
kilosort.run_kilosort: Preprocessing filters computed in 0.70s; total 0.70s
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage after preprocessing
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:     6.80 %
kilosort.run_kilosort: Mem used:     15.10 %     |       9.35 GB
kilosort.run_kilosort: Mem avail:    52.75 / 62.10 GB
kilosort.run_kilosort: ------------------------------------------------------
kilosort.run_kilosort: GPU usage:    `conda install pynvml` for GPU usage
kilosort.run_kilosort: GPU memory:    3.10 %     |      0.45   /    14.56 GB
kiloso

kilosort4 run time 163.24s


In [26]:
import sys, numpy, scipy, scipy.special as sp
print(sys.executable)
print("numpy", numpy.__version__)
print("scipy", scipy.__version__)
print("has btdtr?", hasattr(sp, "btdtr"))


/home/ubuntu/miniconda3/envs/sorter/bin/python
numpy 1.26.4
scipy 1.16.2
has btdtr? True
